This is a tutorial to explain how to use some of the "new" MAF interface via python. 

Using MAF within python can be simpler than using the full driver script + configuration files in the case of a user who wants to use a few metrics or a few sql constraints. I'll try to show the advantages and limitations here, but will mostly focus on the use case example of: <br>
"I am an opsim user who wants to understand the distribution of sky position angles (rotSkyPos in the opsim output tables) in an opsim run" 

I will assume the user has sims_maf installed (see [installation instructions](https://confluence.lsstcorp.org/display/SIM/Catalogs+and+MAF)) and setup (see [setup instructions](https://confluence.lsstcorp.org/display/SIM/MAF+Installation+and+Initial+Examples#MAFInstallationandInitialExamples-runningexamples)).  Let's load all of the MAF modules we'll need. 

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.sliceMetrics as sliceMetrics
from lsst.sims.maf.db import OpsimDatabase
from lsst.sims.maf.utils import opsimUtils

---
The first thing that is required is to break down the question of "understand the distribution of rotSkyPos in opsim" into more specific chunks. What do we want to measure, and over what subset of visits do we want to calculate those measurements? In MAF these map to specific "[metrics](https://confluence.lsstcorp.org/display/SIM/MAF+Metrics)" (how to calculate what we want to measure) and "[slicers](https://confluence.lsstcorp.org/display/SIM/MAF+Slicers)" (how to subdivide the opsim visits).

Let's first address what subset of visits (or subset of the opsim data) we want to use in our measurements. We probably want to look at 'the distribution of rotSkyPos' at a set of RA/Dec points across the sky. That means we can either use a HealpixSlicer (healpix grid of ra/dec points) or an OpsimFieldSlicer (ra/dec points corresponding to the centers of opsim fields). 

In this case, I could see using either one .. but let's use a HealpixSlicer, under the assumption that resolution matters and 'rotSkyPos' makes sense to evaluate even beyond the center of each field.

In [13]:
slicer = slicers.HealpixSlicer(nside=64)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


And now let's think about what we want to measure about the rotSkyPos values at each opsim pointing. Let's start with the mean, rms and full range (max-min).  I'll create these as a list just to make it easier to cart around later. 

In [14]:
metriclist = []
metriclist.append(metrics.MeanMetric('rotSkyPos'))
metriclist.append(metrics.RmsMetric('rotSkyPos'))
metriclist.append(metrics.FullRangeMetric('rotSkyPos'))

We use these to get a "RunSliceMetric" set up. A RunSliceMetric object combines a slicer and metrics, and provides convenience methods to calculate, store, save and visualize the metric data. 

In [15]:
outDir = 'rotSkyPos_Test'
sm = sliceMetrics.RunSliceMetric(outDir=outDir)
sm.setMetricsSlicerStackers(metriclist, slicer)

Okay, so let's connect to an opsim sqlite database and get some data! (some opsim databases are available [here](https://confluence.lsstcorp.org/display/SIM/OpSim+Datasets+for+Cadence+Workshop+LSST2015))

In [16]:
runName = 'enigma_1189'
opsdb = OpsimDatabase('sqlite:///' + runName + '_sqlite.db')

MAF provides convenience methods + built-in framework utilities to identify what data you need from the database (and what could be added 'on-the-fly' using [stackers](https://confluence.lsstcorp.org/display/SIM/MAF+Stackers)).  We'll now use one of those to easily get a list of the columns we'd need from the database (Hint: these columns are 'rotSkyPos', obviously, and 'fieldRA', 'fieldDec' so that the slicer can work and create plots). 

Note that if stackers are necessary for a particular column required anywhere (e.g. if you were calculating MeanMetric on HA, which is not a standard part of the opsim output table), they will automatically be added to the RunSliceMetric. However, users can configure stackers and add them manually, if desired.

In [17]:
dbcols  =sm.findDataCols()
print dbcols

set(['fieldDec', 'fieldRA', 'rotSkyPos'])


Let's get the actual opsim data. We'll just look at r band visits. 

In [18]:
sqlconstraint = '%s' %("filter = 'r'")
simdata = opsimUtils.getSimData(opsdb, sqlconstraint, dbcols, sm.stackerObjs)
print simdata.dtype.names

('obsHistID', 'fieldDec', 'fieldRA', 'rotSkyPos')


And calculate the metric data. 

In [19]:
sm.runSlices(simdata, simDataName=runName, sqlconstraint=sqlconstraint)

Let's save it to disk (we can read this back later). 

In [20]:
sm.writeAll()